In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))
display(HTML("<style>body {font-family: \"D2Coding\", cursive, sans-serif;}</style>"))

In [3]:
#import keras
import os, sys
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.models import Model, load_model
from keras.layers import Dense, Conv1D, MaxPooling1D, GlobalMaxPool1D, BatchNormalization, Dropout, Activation, Add, Layer, GlobalAveragePooling1D, Input, Concatenate,Reshape, Dense, multiply, add, Permute, Lambda
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from keras import optimizers
from keras.utils.vis_utils import plot_model
import pandas as pd
import numpy as np
import random

from sklearn import preprocessing
from sklearn.metrics import auc, classification_report, confusion_matrix, accuracy_score, roc_curve, roc_auc_score, f1_score, precision_recall_curve

In [4]:
BASE_DIR = "./"
FILE_TRAIN_CASES = "./train_cases.csv"    
    
df_tc = pd.read_csv(FILE_TRAIN_CASES)
df_tc.head(10)

,caseid,age,sex,weight,height
0,1,77,M,67.50,160.2
1,4,74,M,53.00,160.6
2,7,52,F,62.30,167.7
3,10,72,M,62.75,162.5
4,13,67,F,64.90,153.0
5,16,57,M,68.90,162.3
6,17,85,M,53.00,164.2
7,19,74,M,66.20,171.3
8,20,75,M,61.30,173.6
9,22,53,F,54.60,162.0


In [5]:
def moving_average(a, n=200):
    '''
    각각 size=n 만큼의 윈도우의 평균값을 갖게한다.
    '''
    ret = np.nancumsum(a, dtype=np.float32) # 누적합 배열을 만든다.        
    '''
    ret[:-n] 처음부터 시작, 끝에서 n개 뺀 것까지. (0 ~ len(arr)-n)
    
    ret[n:] = ret[n:] - ret[:-n]
    assume n = 5 :
      5 6 7 8 9 10 11 12 13 ...
    - 0 1 2 3 4  5  6  7  8 ...
    ------------------------
      5 5 5 5 5  5  5 ....

    여기서 ret가 누적합이므로, moving average값이 된다.
    '''
    ret[n:] = ret[n:] - ret[:-n] # window에 포함된 값들의 총합들을 구한다. n개의 합부터 시작
    return ret[n - 1:] / n

a = [i for i in range(20)]
print(a)
print(moving_average(a, n=10))




[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]
[ 4.5  5.5  6.5  7.5  8.5  9.5 10.5 11.5 12.5 13.5 14.5]


In [6]:
import os
import pandas as pd
import gc

DATA_VERSION = 14

sample_rate = 100

MINUTES_AHEAD = 5
BATCH_SIZE = 512


def make_filename(typename, ext):
    ret = typename + '_' + str(DATA_VERSION) + "_" + str(random_keyval) + '.' + ext
    ret = os.path.join(BASE_DIR, ret)
    return ret


# As provided in the answer by Divakar
def ffill(arr):
    mask = np.isnan(arr)
    idx = np.where(~mask, np.arange(mask.shape[1]), 0)
    np.maximum.accumulate(idx, axis=1, out=idx)
    out = arr[np.arange(idx.shape[0])[:,None], idx]
    return out


# My modification to do a backward-fill
def bfill(arr):
    mask = np.isnan(arr)
    idx = np.where(~mask, np.arange(mask.shape[1]), mask.shape[1] - 1)
    idx = np.minimum.accumulate(idx[:, ::-1], axis=1)[:, ::-1]
    out = arr[np.arange(idx.shape[0])[:,None], idx]
    return out


def fb_fill_1dim(arr):
    arr = arr.reshape((1, -1))
    arr = ffill(arr)
    arr = bfill(arr)
    arr = arr.reshape((-1))
    return arr

def min_to_sec(minval):
    return minval * 60

def to_sr(a):
    return sample_rate * a

in_wlen = to_sr(20)

def calc_hb_hr(segx):
    """extrace number of heartbeats in segment and heartbeat rate"""
    
    prominence_candidates = [20, 16, 12, 10]
    for prominence in prominence_candidates:
        peaks, _ = find_peaks(segx, distance=50, prominence=prominence)    
        heartbeats = len(peaks)
        diffs = np.diff(peaks)
        if len(diffs) == 0 or heartbeats == 0:
            continue            
        heartrates = round(in_wlen / np.diff(peaks).mean())
        return heartbeats, heartrates
    return 0, 0


def check_valid(segx):    
    if np.mean(np.isnan(segx)) > 0.1 or \
        np.max(segx) > 200 or np.min(segx) < 20 or \
        np.max(segx) - np.min(segx) < 30 or \
        (np.abs(np.diff(segx[~np.isnan(segx)])) > 30).any():
        return False
    return True



In [7]:
# extract demographic data from test dataset
if os.path.exists("x_test_cases.csv") == False:
    x_test = pd.read_csv(os.path.join(BASE_DIR, 'test2_x.csv'))    
    x_test_cases = x_test[['age', 'sex', 'weight', 'height']]
    x_test_cases.to_csv("x_test_cases.csv", index=False)

    
def get_scaler():
    """generator에서 데이터 생성할 때마다 사용할 scaler를 생성함"""
    
    x_train_cases = pd.read_csv(FILE_TRAIN_CASES)
    x_train_cases = x_train_cases.drop('caseid', axis=1)
    x_test_cases = pd.read_csv("x_test_cases.csv")
    tmp = pd.concat([x_train_cases, x_test_cases])
    train_len = x_train_cases.shape[0]
    tmp = pd.get_dummies(tmp)
    scaler = preprocessing.MinMaxScaler()
    print(tmp.shape)
    tmp = scaler.fit_transform(tmp)    
    print(tmp[0])
    print(tmp.shape)
    return scaler


demographic_info_scaler = get_scaler()

(102281, 5)
[0.77631579 0.33882784 0.46007605 0.         1.        ]
(102281, 5)


In [23]:
def build_one_epoch(scaler, frac=0.5):
    gc.collect()

    x_train = []
    x_train_cases = []
    y_train = []
    info_columns = ['caseid', 'age', 'sex', 'weight', 'height']
    hb_err_cnt = 0

    df_tc = pd.read_csv(FILE_TRAIN_CASES)    
    df_tc = df_tc.sample(frac=frac)
        
    num_processed = 0
    for idx, row in df_tc.iterrows():
        caseid = row['caseid']
        age = row['age']
        sex = row['sex']
        weight = row['weight']
        height = row['height']
        
        num_processed += 1
        
        b = "Building 1-epoch Data " + str(int((num_processed * 100)/df_tc.shape[0])) + "%"
        sys.stdout.write('\r'+ b)

        fpath = os.path.join(BASE_DIR, ''.join(['train_data/', str(caseid), '.csv']))
        if os.path.isfile(fpath):
            samples = pd.read_csv(fpath, header=None).values.flatten() # 읽은값이 (x, 1) 이어서 1차원으로 바꿔준다.

            # 20sec (20 00) - 5min (300 00) - 1min (60 00) = 38000 sample
            
            i = random.randint(to_sr(10), to_sr(40)) # random start

            event_idx = []
            non_event_idx = []

            yidx_start = to_sr(20 + min_to_sec(MINUTES_AHEAD)) # output 크기
            yidx_end = yidx_start + to_sr(min_to_sec(1))
            while i < (len(samples) - yidx_end) :                
                segx = samples[i : i + in_wlen] # 20sec samples window
                segy = samples[i + yidx_start : i + yidx_end] # 출력 : 20초 후부터 5분 후 1분동안의 MAP값
                
                if (check_valid(segx) == False) or (check_valid(segy) == False):
                    i += random.randint(to_sr(1), to_sr(10)) # random move
                    continue

                segx = fb_fill_1dim(segx) # nan 값을 앞뒤 값으로 채운다.

                # demographic data 패키징
                info_pack = [caseid, age, sex, weight, height]

                # 출력변수
                segy = moving_average(segy, to_sr(2)) # 2 sec moving average

                # event 여부 확인
                event = 1 if np.nanmax(segy) < 65 else 0

                # train 데이터 추가
                event_idx.append(i) if event else non_event_idx.append(i)
                x_train.append(segx)
                y_train.append(event)
                x_train_cases.append(info_pack)
                
                # move i by random
                i += random.randint(to_sr(5), to_sr(35))

            nsamp = len(event_idx) + len(non_event_idx)

    print()
    x_train = np.array(x_train, dtype=np.float32)
    
    # postprocess x_train value if needed
    x_train -= 65
    x_train /= 65

    y_train = np.array(y_train, dtype=np.float32)
    
    # one-hot encode categorical columns and normalize.
    x_train_cases = pd.DataFrame(x_train_cases, columns = info_columns)
    gc.collect()
    _xtrain_cases = x_train_cases
    if 'caseid' in _xtrain_cases:
        _xtrain_cases = _xtrain_cases.drop('caseid', axis=1)
    _xtest_cases = pd.read_csv("x_test_cases.csv")
    tmp = pd.concat([_xtrain_cases, _xtest_cases])
    _xtrain_cases_len = x_train_cases.shape[0]
    tmp = pd.get_dummies(tmp)
    tmp = scaler.transform(tmp)    
    xtrain_cases = tmp[:_xtrain_cases_len]    

    # shuffle data several times
    shuffle_cnt = random.randint(4, 10)
    for i in range(shuffle_cnt):
        s = np.arange(x_train.shape[0])
        np.random.shuffle(s)
        x_train = x_train[s]
        y_train = y_train[s]
        xtrain_cases = xtrain_cases[s]
    
    # concatenate MBP and demographic data
    # will be split in first layer of model
    x_train = np.concatenate([x_train, xtrain_cases], axis=1)    
    return x_train, y_train   


for i in range(1):
    build_one_epoch(scaler=demographic_info_scaler, frac=0.01)
    

Building 1-epoch Data 100%


In [18]:
class Generator(tf.keras.utils.Sequence):
    # Class is a dataset wrapper for better training performance
    def __init__(self, batch_size=256, frac=0.05):
        self.batch_size = batch_size
        self.frac = frac
        self.on_epoch_end()

    def __len__(self): # return number of batches
        #print(self.x_train.shape[0], self.batch_size)
        num_batch = int(self.x_train.shape[0] / self.batch_size)
        if self.x_train.shape[0] % self.batch_size > 0:
            num_batch += 1
        #print("num batch=", num_batch)
        return num_batch

    def __getitem__(self, idx): # return 1-batch data
        start = idx * self.batch_size
        end = (idx+1) * self.batch_size
        if end > len(self.x_train):
            end = len(self.x_train)
        
        batch_x = self.x_train[start:end]
        batch_y = self.y_train[start:end]
        return batch_x, batch_y

    def on_epoch_end(self):
        x_train, y_train = build_one_epoch(scaler=demographic_info_scaler, frac=self.frac)
        self.x_train = x_train        
        self.y_train = y_train
        
        

In [13]:
def squeeze_excite_block(input_tensor, ratio=16):
    """ Create a channel-wise squeeze-excite block
    Args:
        input_tensor: input Keras tensor
        ratio: number of output filters
    Returns: a Keras tensor
    References
    -   [Squeeze and Excitation Networks](https://arxiv.org/abs/1709.01507)
    """
    init = input_tensor
    channel_axis = 1 if tf.keras.backend.image_data_format() == "channels_first" else -1
    filters = init.shape[channel_axis]
    se_shape = (1, filters)

    se = keras.layers.GlobalAveragePooling1D()(init)
    se = Reshape(se_shape)(se)    
    se = keras.layers.Dense(filters // ratio, activation='relu', kernel_initializer='he_normal', use_bias=False)(se)
    se = keras.layers.Dense(filters, activation='sigmoid', kernel_initializer='he_normal', use_bias=False)(se)    

    x = multiply([init, se])
    return x


def resnet_basic_block(inputs, filters, kernel_size, strides):
    x = inputs
    shortcut = inputs
    print("x.shape=", x.shape)
    x = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=strides, padding='same', activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)
    
    x = keras.layers.Conv1D(filters=filters, kernel_size=kernel_size, strides=1, padding='same', activation='relu')(x)
    x = keras.layers.BatchNormalization()(x)    

    x = squeeze_excite_block(x)
    
    shortcut = keras.layers.Conv1D(filters=filters, kernel_size=1, strides=strides, padding='same', activation='relu')(shortcut)
    shortcut = keras.layers.BatchNormalization()(shortcut)

    x = keras.layers.Add()([shortcut, x])
    x = keras.layers.BatchNormalization()(x)
    x = keras.layers.Activation('relu')(x)
    return x


def resnet(inputs, kernel_size, filter_sizes):
    x = inputs
    for fsize in filter_sizes:
        x = resnet_basic_block(inputs=x, filters=fsize, kernel_size=kernel_size, strides=2)
    return x


def resnet_demo_net(inputs, kernel_sizes, filter_sizes):
    
    # you have to use keras layer class for splitting.
    # just splitting like array doesn't work    
    
    # bp data
    input_bp = Lambda(lambda x: x[:, :2000])(inputs)
    input_bp = tf.keras.backend.expand_dims(input_bp, axis=-1)
    
    # demographic
    input_demo = Lambda(lambda x: x[:, 2000:])(inputs)

    sub_models = []
    for kernel_size in kernel_sizes:        
        sm = resnet(input_bp, kernel_size=kernel_size, filter_sizes=filter_sizes)
        sm = keras.layers.GlobalAveragePooling1D()(sm)        
        sub_models.append(sm)

    if len(sub_models) > 1:
        #x = keras.layers.Add()(sub_models)
        x = keras.layers.Concatenate()(sub_models)
    else:
        x = sub_models[0]
    
    x = Concatenate()([x, input_demo])

    x = tf.expand_dims(x, -1)

    x = resnet_basic_block(x, filters=16, kernel_size=3, strides=2)
    x = resnet_basic_block(x, filters=32, kernel_size=3, strides=2)
    x = resnet_basic_block(x, filters=64, kernel_size=3, strides=2)
    
    x = keras.layers.GlobalAveragePooling1D()(x)
    x = keras.layers.Dropout(0.3)(x)
    x = keras.layers.Dense(units=1, activation='sigmoid')(x)

    return x

input_tensor = Input(shape=(2005,), dtype='float32')

kernel_sizes = [3]
filter_sizes = [32, 32, 32, 64, 64, 64, 128, 128]

def prepare_model():
    tf.keras.backend.clear_session()
    mergenet = resnet_demo_net(input_tensor, kernel_sizes=kernel_sizes, filter_sizes=filter_sizes)
    mergenet_model = keras.models.Model(inputs=input_tensor, outputs=mergenet)
    mergenet_model.compile(loss='binary_crossentropy', optimizer=optimizers.Adam(), metrics=['accuracy', tf.keras.metrics.AUC(name="auc_mergenet")])
    return mergenet_model

mergenet_model = prepare_model()
plot_model(mergenet_model, to_file='lowprenet.png', show_shapes=True, show_layer_names=True)

x.shape= (None, 2000, 1)
x.shape= (None, 1000, 32)
x.shape= (None, 500, 32)
x.shape= (None, 250, 32)
x.shape= (None, 125, 64)
x.shape= (None, 63, 64)
x.shape= (None, 32, 64)
x.shape= (None, 16, 128)
x.shape= (None, 133, 1)
x.shape= (None, 67, 16)
x.shape= (None, 34, 32)
('Failed to import pydot. You must `pip install pydot` and install graphviz (https://graphviz.gitlab.io/download/), ', 'for `pydotprint` to work.')


In [24]:
EPOCHS = 1
BATCH_SIZE = 512

def make_callbacks(weight_path):
    callbacks = []    
    callbacks.append(ModelCheckpoint(monitor='loss', filepath=weight_path, verbose=1, save_best_only=True))
    #callbacks.append(ReduceLROnPlateau(monitor = 'val_loss', factor = 0.1, patience=2, min_lr=0.00001, verbose=1, mode='min'))    
    callbacks.append(EarlyStopping(monitor='val_auc_mergenet', patience=1, verbose=0, mode='max', min_delta=0.015))
    return callbacks


def train_model(model, weight_path, saving_model_path, batch_size, epochs):    

    callbacks = make_callbacks(weight_path)
    
    train_gen = Generator(BATCH_SIZE, frac=0.01)
    valid_gen = Generator(BATCH_SIZE, frac=0.01)
    
    hist = model.fit(train_gen, validation_data=valid_gen, batch_size=batch_size, class_weight={0:1, 1:10}, callbacks=callbacks, epochs=epochs)
    
    open(saving_model_path, "wt").write(model.to_json())   

    return model


# # train mbp-only model
model_ver = 6
model_name = "mergenet_model"
outdir = "se_resnet_deep_end_resnet_generator_dataset" + "_" + str(model_ver)
weight_path = os.path.join(BASE_DIR, outdir, model_name + "weights.hdf5")
os.makedirs(os.path.dirname(weight_path), exist_ok=True)
saving_model_path = os.path.join(BASE_DIR, outdir, model_name + "model.json")


model = train_model(mergenet_model, weight_path, saving_model_path, BATCH_SIZE, EPOCHS)

Building 1-epoch Data 100%
Building 1-epoch Data 100%
Building 1-epoch Data 100%===========] - ETA: 0s - loss: 0.3811 - accuracy: 0.9293 - auc_mergenet: 0.8254

Epoch 00001: loss improved from inf to 0.38113, saving model to ./se_resnet_deep_end_resnet_generator_dataset_6\mergenet_modelweights.hdf5
33/33 [==============================] - 90s 3s/step - loss: 0.3811 - accuracy: 0.9293 - auc_mergenet: 0.8254 - val_loss: 1024453443584.0000 - val_accuracy: 0.9486 - val_auc_mergenet: 0.8676
Building 1-epoch Data 100%


In [29]:
    # prepare test data  
    x_test = []
    x_test_cases = None
    
    x_test = pd.read_csv(os.path.join(BASE_DIR, 'test2_x.csv'))    
    x_test = x_test.values
    x_test = np.array(x_test[:, 4:], dtype=np.float32) # skip demographic values(age, sex weight, height)
    x_test = ffill(x_test)
    x_test = bfill(x_test)
    x_test -= 65
    x_test /= 65
    
    x_test_cases = pd.read_csv("x_test_cases.csv")
    x_test_cases = pd.get_dummies(x_test_cases)
    xtest_cases = demographic_info_scaler.transform(x_test_cases)    

In [35]:
x_test_input = np.concatenate([x_test, xtest_cases], axis=1)

y_pred = model.predict(x = x_test_input).flatten()

def savepredfile(filename, pred):
    savefilename = os.path.join(BASE_DIR, "preds")
    savefilename = os.path.join(savefilename, filename)
    np.savetxt(savefilename, pred, fmt="%.4f")
    np.savetxt(filename, pred, fmt="%.4f")

savepredfile("pred_y.txt", y_pred)